In [5]:
#Cargamos el dataframe

import pandas as pd

df = pd.read_csv('./data/unique_ai_tweets_with_issues.csv')

print(df.head())


# ¿Qué debería tener cada columna?
#Tenemos las columnas: Username Full_Name Content Created_at Retweet Likes Replies Verificado

#Username: String no nulo
#Full_Name: String
#Content: String, no nulo
#Created_at: Datetime, no nulo, Limite superior la fecha actual. Limite infeior 2006.
#Retweets like replies: int
#Verificado: Boolean

df_schema = pa.DataFrameSchema({
    "Survived": pa.Column(pa.Int, pa.Check.in_range(0, 1)),
    "Pclass": pa.Column(pa.Int, nullable=True),
    "Name": pa.Column(pa.String, nullable=True),
    "Sex": pa.Column(pa.String, pa.Check.isin(["male", "female"])),
    "Age": pa.Column(pa.Float, pa.Check(lambda x: x > 20), nullable=False),
    "Siblings/Spouses Aboard": pa.Column(pa.Int, nullable=True),
    "Parents/Children Aboard": pa.Column(pa.Int, nullable=True),
    "Fare": pa.Column(pa.Float, pa.Check(lambda x: x >= 0), nullable=True)
})


          username       full_name  \
0         carrie86    Mark Pacheco   
1        murphyann     Olivia Hill   
2  williamskenneth  Cindy Anderson   
3  martinezdeborah     Mark Hunter   
4         pstewart    Sara Gregory   

                                             content           created_at  \
0  Board knowledge agent scientist opportunity la...  2024-08-19 17:36:21   
1  Sometimes first phone unit various draw board ...  2024-08-20 07:44:20   
2  Him social way ability debate serve he conditi...  2024-07-21 16:24:56   
3  Walk serve water listen determine wrong level ...  2024-06-06 21:20:35   
4  Usually study soldier since young important wi...  2024-09-10 01:48:40   

  retweets likes replies verified  
0      929  1019     116    False  
1      663  4966      86    False  
2      506   835     257    False  
3      762  2008     397    False  
4      729  3821     203    False  
